In [47]:
!pip install pyspark

In [48]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType
import matplotlib.pyplot as plt


In [49]:
spark = SparkSession.builder.getOrCreate()

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
#dataframe
df_ipca = spark.read.csv('/content/drive/MyDrive/Projeto de Parceria  Semantix/Ipca_final.csv', header=True, inferSchema=True)
df_selic = spark.read.csv('/content/drive/MyDrive/Projeto de Parceria  Semantix/selic_final.csv', header=True, inferSchema=True)

In [52]:
df_ipca.show(5)

+----+---+-----------+---------------+-------------------+------------------+--------------+-------------------+----------+--------------------+---------------+
| ano|mes|     indice|variacao_mensal|variacao_trimestral|variacao_semestral|variacao_anual|variacao_doze_meses|      data|   periodo_economico|ciclo_economico|
+----+---+-----------+---------------+-------------------+------------------+--------------+-------------------+----------+--------------------+---------------+
|2000|  1|    1598.41|           0.62|               2.19|               4.3|          0.62|               8.85|2000-01-01|Consolidação do Real|       Expansão|
|2016|  1|    4550.23|           1.27|               3.27|              4.91|          1.27|              10.71|2016-01-01|     Crise Econômica|      Contração|
|1985|  1|5.656761E-7|          11.76|              38.32|             86.66|         11.76|             221.27|1985-01-01|   Pré-Plano Cruzado|      Contração|
|1997|  1|    1379.33|           1

In [53]:
df_selic.show(5)

+----+--------------+-------+---+---------+---------+-----------------+---------------+
| ano|data_formatada|   Data|mes|selic_mes|selic_ano|periodo_economico|ciclo_economico|
+----+--------------+-------+---+---------+---------+-----------------+---------------+
|1996|    1996-07-01|1996.07|  7|     NULL|  22.9988|       Plano Real|       Expansão|
|1996|    1996-08-01|1996.08|  8|  25.3401|  22.9988|       Plano Real|       Expansão|
|1996|    1996-09-01|1996.09|  9|  25.0453|  22.9988|       Plano Real|       Expansão|
|1996|    1996-10-01| 1996.1| 10|  24.1644|  22.9988|       Plano Real|       Expansão|
|1996|    1996-11-01|1996.11| 11|  23.5803|  22.9988|       Plano Real|       Expansão|
+----+--------------+-------+---+---------+---------+-----------------+---------------+
only showing top 5 rows



In [54]:
df_selic.printSchema()
df_ipca.printSchema()

root
 |-- ano: integer (nullable = true)
 |-- data_formatada: date (nullable = true)
 |-- Data: double (nullable = true)
 |-- mes: integer (nullable = true)
 |-- selic_mes: double (nullable = true)
 |-- selic_ano: double (nullable = true)
 |-- periodo_economico: string (nullable = true)
 |-- ciclo_economico: string (nullable = true)

root
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- indice: double (nullable = true)
 |-- variacao_mensal: double (nullable = true)
 |-- variacao_trimestral: double (nullable = true)
 |-- variacao_semestral: double (nullable = true)
 |-- variacao_anual: double (nullable = true)
 |-- variacao_doze_meses: double (nullable = true)
 |-- data: date (nullable = true)
 |-- periodo_economico: string (nullable = true)
 |-- ciclo_economico: string (nullable = true)



In [55]:
# maxima e minima do IPCA
df_ipca.select(
    F.max('variacao_anual').alias('max_anual'),
    F.min('variacao_anual').alias('min_anual'),
    F.max('variacao_mensal').alias('max_mensal'),
    F.min('variacao_mensal').alias('min_mensal'),
    F.max('variacao_trimestral').alias('max_trimestral'),
    F.min('variacao_trimestral').alias('min_trimestral'),
    F.max('variacao_semestral').alias('max_semestral'),
    F.min('variacao_semestral').alias('min_semestral')
).show()


+---------+---------+----------+----------+--------------+--------------+-------------+-------------+
|max_anual|min_anual|max_mensal|min_mensal|max_trimestral|min_trimestral|max_semestral|min_semestral|
+---------+---------+----------+----------+--------------+--------------+-------------+-------------+
|  2477.15|    -0.16|     82.39|     -0.68|        437.02|         -1.32|      1580.94|        -0.93|
+---------+---------+----------+----------+--------------+--------------+-------------+-------------+



In [56]:
#medina ipca
mediana_anual = df_ipca.approxQuantile('variacao_anual', [0.5], 0.01)[0]
mediana_mensal = df_ipca.approxQuantile('variacao_mensal', [0.5], 0.01)[0]
mediana_trimestral = df_ipca.approxQuantile('variacao_trimestral', [0.5], 0.01)[0]
mediana_semestral = df_ipca.approxQuantile('variacao_semestral', [0.5], 0.01)[0]

print(f'Mediana Anual: {mediana_anual}')
print(f'Mediana Mensal: {mediana_mensal}')
print(f'Mediana Trimestral: {mediana_trimestral}')
print(f'Mediana Semestral: {mediana_semestral}')


Mediana Anual: 4.76
Mediana Mensal: 0.7
Mediana Trimestral: 2.04
Mediana Semestral: 3.88


In [57]:
#maxima e minima selic
df_selic.select(
    F.max('selic_mes').alias('max_mes'),
    F.min('selic_mes').alias('min_mes'),
    F.max('selic_ano').alias('max_ano'),
    F.min('selic_ano').alias('min_ano')
).show()

+-------+-------+-------+-------+
|max_mes|min_mes|max_ano|min_ano|
+-------+-------+-------+-------+
|43.4089|    2.0|40.9238|    2.0|
+-------+-------+-------+-------+



In [ ]:
#mediana selic
mediana_anual = df_selic.approxQuantile('selic_ano', [0.5], 0.01)[0]
mediana_mensal = df_selic.approxQuantile('selic_mes', [0.5], 0.01)[0]

print(f'Mediana Anual: {mediana_anual}')
print(f'Mediana Mensal: {mediana_mensal}')

In [ ]:

#Inflação ao longo dos últimos anos

# Filtrar apenas os dados de dezembro
df_dezembro = df_ipca.filter(F.col('mes') == 12)


df_ipca_ano = df_dezembro.select('ano', 'variacao_mensal').withColumnRenamed('variacao_mensal', 'ipca_ano').orderBy('ano')

df_plot = df_ipca_ano.toPandas()



plt.figure(figsize=(10,6))
plt.bar(df_plot['ano'], df_plot['ipca_ano'], color='royalblue')
plt.title('Inflação ao longo dos últimos anos (%)')
plt.xlabel('Ano')
plt.ylabel('IPCA (%)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()



In [ ]:
#selic ao longo dos últimos anos

# Filtrar apenas os dados de dezembro
df_dezembro = df_selic.filter(F.col('mes') == 12)

# Selecionar ano e selic acumulado (que está na variacao_mensal ou outra coluna de interesse)
df_selic_ano = df_dezembro.select('ano', 'selic_mes').withColumnRenamed('selic_mes', 'selic_ano').orderBy('ano')

df_plot = df_selic_ano.toPandas()



plt.figure(figsize=(10,6))
plt.bar(df_plot['ano'], df_plot['selic_ano'], color='royalblue')
plt.title('Selic ao longo dos últimos anos (%)')
plt.xlabel('Ano')
plt.ylabel('Selic ano (%)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()


In [ ]:
#mediana ipca mes

#lista
meses = list(range(1,13))
medianas=[]

#loop para mediana de cada mes
for mes in meses:
    mediana = df_ipca.filter(F.col('mes') == mes) \
        .approxQuantile('variacao_mensal', [0.5], 0.01)[0]
    medianas.append((mes, mediana))

# Exibir o resultado
for mes, valor in medianas:
    print(f'Mês {mes}: Mediana = {valor}')



In [ ]:
#mediana selic

#lista
meses = list(range(1,13))
medianas=[]

#loop para mediana de cada mes
for mes in meses:
    mediana = df_selic.filter(F.col('mes') == mes) \
        .approxQuantile('selic_mes', [0.5], 0.01)[0]
    medianas.append((mes, mediana))

# Exibir o resultado
for mes, valor in medianas:
    print(f'Mês {mes}: Mediana = {valor}')




In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import Window

# Filtra desde 1994
df_ipca_1994 = df_ipca.filter(F.col("ano") >= 1994)

# Agrupa por ano e soma variacao_mensal
df_anual = df_ipca_1994.groupBy("ano").agg(
    F.sum("variacao_mensal").alias("soma_variacao_anual")
).orderBy("ano")

# Define uma janela ordenada por ano para calcular o acumulado
windowSpec = Window.orderBy("ano").rowsBetween(Window.unboundedPreceding, 0)

# Calcula o acumulado da soma da variação anual
df_anual = df_anual.withColumn(
    "soma_acumulada",
    F.sum("soma_variacao_anual").over(windowSpec)
)

# Mostra o resultado
df_anual.show(30)



In [ ]:
#contar periodo economico e ciclo economico
df_ipca.groupBy("periodo_economico").count().show()
df_ipca.groupBy("ciclo_economico").count().show()

In [ ]:
# max,min e mediana de ciclos economicos ipca

df_ipca.groupBy("ciclo_economico").agg(
    max("variacao_anual").alias("IPCA_ANUAL_MAXIMA"),
    min("variacao_anual").alias("IPCA_ANUAL_MINIMA"),
    percentile_approx("variacao_anual", 0.5).alias("IPCA_ANUAL_MEDIANA")
).show()


In [ ]:
# max,min e mediana de ciclos economicos selic

df_selic.groupBy("ciclo_economico").agg(
    max("selic_ano").alias("SELIC_ANUAL_MAXIMA"),
    min("selic_ano").alias("SELIC_ANUAL_MINIMA"),
    percentile_approx("selic_ano", 0.5).alias("SELIC_ANUAL_MEDIANA")
).show()


In [ ]:
# max,min e mediana de periodo economico ipca

df_ipca.groupBy("periodo_economico").agg(
    max("variacao_anual").alias("IPCA_ANUAL_MAXIMA"),
    min("variacao_anual").alias("IPCA_ANUAL_MINIMA"),
    percentile_approx("variacao_anual", 0.5).alias("IPCA_ANUAL_MEDIANA")
).show()


In [ ]:
# max,min e mediana de periodo economico selic

df_selic.groupBy("periodo_economico").agg(
    max("selic_ano").alias("SELIC_ANUAL_MAXIMA"),
    min("selic_ano").alias("SELIC_ANUAL_MINIMA"),
    percentile_approx("selic_ano", 0.5).alias("SELIC_ANUAL_MEDIANA")
).show()
